In [ ]:
# poker

In [1]:
from itertools import product
from random import sample

ranks = '2 3 4 5 6 7 8 9 10 J Q K A'.split()
suits = 'S H C D'.split()
deck = [r+s for r, s in product(ranks, suits)]

In [23]:
from collections import defaultdict

def rank_and_suit_dicts(hand):
    ranks = '2 3 4 5 6 7 8 9 10 J Q K A'.split()
    rank_val_dict = dict(zip(ranks, range(13)))
    rbs = defaultdict(list)
    sbr = defaultdict(list)
    for card in hand:
        rank, suit = rank_val_dict[card[:-1]], card[-1]
        rbs[suit].append(rank)
        sbr[rank].append(suit)
    return rbs, sbr


def is_sequential(ranklist):
    return max(ranklist) - min(ranklist) == len(ranklist) - 1


def score(hand):
    rbs, sbr = rank_and_suit_dicts(hand)
    suit_counts = sorted(map(len,rbs.values()))
    rank_counts = sorted(map(len,sbr.values()))
    kicker = 0
    # straight flush?
    if suit_counts == [5] and is_sequential(list(sbr.keys())):
        hand_score = 9
        rank = max(sbr.keys())
    # four of a kind?
    elif rank_counts == [1, 4]:
        hand_score = 8
        rank = [k for k, v in sbr.items() if len(v) == 4][0]
        kicker = [k for k, v in sbr.items() if len(v) == 1][0]
    # full house?
    elif rank_counts == [2, 3]:
        hand_score = 7
        triplet_score = [k for k, v in sbr.items() if len(v) == 3][0]
        pair_score = [k for k, v in sbr.items() if len(v) == 2][0]
        rank = 13 * triplet_score + pair_score
    # flush?
    elif suit_counts == [5]:
        hand_score = 6
        rank = max(sbr.keys())
    # straight?
    elif is_sequential(sbr.keys()):
        hand_score = 5
        rank = max(sbr.keys())
    # three of a kind?
    elif rank_counts == [1, 1, 3]:
        hand_score = 4
        rank = [k for k, v in sbr.items() if len(v) == 3][0]
        kicker = max([k for k, v in sbr.items() if len(v) == 1])
    # two pair?
    elif rank_counts == [1, 2, 2]:
        hand_score = 3
        pair_scores = [k for k, v in sbr.items() if len(v) == 2]
        rank = max(pair_scores)
        low_pair_score = min(pair_scores)
        single_card_score = [k for k, v in sbr.items() if len(v) == 1][0]
        kicker = 13 * low_pair_score + single_card_score
    # pair?
    elif rank_counts == [1, 1, 1, 2]:
        hand_score = 2
        rank = [k for k, v in sbr.items() if len(v) == 2][0]
        k3, k2, k1 = sorted([k for k, v in sbr.items() if len(v) == 1])
        kicker = (13**2 * k1) + (13 * k2) + k3 
    # high card
    else:
        hand_score = 1
        k4, k3, k2, k1, high = sorted(sbr.keys())
        rank = high
        kicker = (13**3 * k1) + (13**2 * k2) + (13 * k3) + k4 
    return hand_score, rank, kicker


def poker(hands):
    scored_hands = sorted([(score(hand), hand) for hand in hands])
    return [hand 
            for score, hand in scored_hands 
            if score == scored_hands[-1][0]]

In [24]:
fourOf3low = '3S 3H 2S 3D 3C'.split()
fourOf3high = '3S 3H 4S 3D 3C'.split()

score(fourOf3high), score(fourOf3low)

((8, 1, 2), (8, 1, 0))

In [7]:
sflush = '8S 9S 10S JS QS'.split()
foak = '4S 4C 4D 4H 10H'.split()
fullh = '3S 3D 5D 5H 5C'.split()
flush = '2C 4C 7C JC AC'.split()
straight = '3S 4H 5D 6H 7C'.split()
throak = '2S 4C 4D 4H 10H'.split()
twop = '8S 8D 5D 5H JC'.split()
pair = '3S 4S 4D 6H 7C'.split()
high = '2S 4S 5D 8H JH'.split()

In [19]:
rand = sample(deck, 5)
rand2 = sample(deck, 5)
poker([rand, rand2, twop, pair])

[['8S', '8D', '5D', '5H', 'JC']]

9: straight flush: sequential ranks, same suit
8: four of a kind: four of same rank, kicker wins tie
7: full house: three of a kind, and one pair
6: flush: five cards of suit, non-sequential
5: straight: five sequential cards of differing suits
4: three of a kind: kickers break tie
3: two pair: kbt
2: one pair: 
1: high card:

In [ ]:
# tree-building

In [3]:
class Record():
    def __init__(self, record_id, parent_id):
        self.record_id = record_id
        self.parent_id = parent_id


class Node():
    def __init__(self, node_id):
        self.node_id = node_id
        self.children = []


def BuildTree(records):
    root = None
    records.sort(key=lambda x: x.record_id)
    ordered_id = [i.record_id for i in records]
    if records:
        if ordered_id[-1] != len(ordered_id) - 1:
            raise ValueError
        if ordered_id[0] != 0:
            raise ValueError
    trees = []
    parent = {}
    for i in range(len(ordered_id)):
        for j in records:
            if ordered_id[i] == j.record_id:
                if j.record_id == 0:
                    if j.parent_id != 0:
                        raise ValueError
                if j.record_id < j.parent_id:
                    raise ValueError
                if j.record_id == j.parent_id:
                    if j.record_id != 0:
                        raise ValueError
                trees.append(Node(ordered_id[i]))
    for i in range(len(ordered_id)):
        for j in trees:
            if i == j.node_id:
                parent = j
        for j in records:
            if j.parent_id == i:
                for k in trees:
                    if k.node_id == 0:
                        continue
                    if j.record_id == k.node_id:
                        child = k
                        parent.children.append(child)
    if len(trees) > 0:
        root = trees[0]
    return root


In [4]:
def find_parent(root, p_id):
    to_visit = [root]
    while to_visit:
        node = to_visit.pop()
        if node.node_id == p_id:
            return node
        else:
            to_visit.extend(node.children)
    raise ValueError

    
def BuildTree(records):
    if not records:
        return None
    if not sorted([r.record_id for r in records]) == list(range(len(records))):
        raise ValueError
    records.sort(key=lambda x: (x.parent_id, x.record_id))
    root = Node(0)
    for rec in records[1:]:
        if rec.parent_id >= rec.record_id:
            raise ValueError
        node = find_parent(root, rec.parent_id)
        node.children.append(Node(rec.record_id))
    return root

In [6]:
records = [
            Record(6, 2),
            Record(0, 0),
            Record(3, 1),
            Record(2, 0),
            Record(4, 1),
            Record(5, 2),
            Record(1, 0)
        ]

root = BuildTree(records)

In [ ]:
# transpose

In [32]:
from itertools import zip_longest


def transpose(input_lines):
    if not input_lines:
        return ''
    inp = [line.replace(' ', '~') for line in input_lines.split('\n')]
    transpose = list(map(lambda z: ''.join(z),
                         zip_longest(*inp, fillvalue=' ')))
    transpose = [line.rstrip().replace('~', ' ') for line in transpose]
    return '\n'.join(transpose)

In [33]:
print(transpose('this\nis\nnot so hard\nis it'))

tini
hsos
i t 
s  i
  st
  o
   
  h
  a
  r
  d


In [ ]:
# dot_ds

In [49]:
NODE, EDGE, ATTR = range(3)


class Node(object):
    def __init__(self, name, attrs={}):
        self.name = name
        self.attrs = attrs

    def __eq__(self, other):
        return self.name == other.name and self.attrs == other.attrs


class Edge(object):
    def __init__(self, src, dst, attrs={}):
        self.src = src
        self.dst = dst
        self.attrs = attrs

    def __eq__(self, other):
        return (self.src == other.src and
                self.dst == other.dst and
                self.attrs == other.attrs)


def validate_graph_data(data):
    if not type(data) == list or min(map(len, data)) < 3:
        raise TypeError
    for k, *d in data:
        dtypes = list(map(type, d))
        if k not in (ATTR, NODE, EDGE):
            raise ValueError
        elif (
                (k == ATTR and dtypes != [str, str]) or
                (k == NODE and not (dtypes == [str, dict] or
                                    dtypes == [str, set])) or
                (k == EDGE and dtypes != [str, str, dict])
             ):
            raise ValueError


class Graph(object):
    def __init__(self, data=[]):
        self.attrs = {}
        self.nodes = []
        self.edges = []
        if data:
            validate_graph_data(data)
            self.assign(data)

    def assign(self, data):
        for k, *d in data:
            if k == ATTR:
                self.attrs[d[0]] = d[1]
            elif k == NODE:
                self.nodes.append(Node(*d))
            elif k == EDGE:
                self.edges.append(Edge(*d))


In [50]:
data = [(ATTR, "foo", "1"),
        (ATTR, "title", "Testing Attrs"),
        (NODE, "a", {"color": "green"}),
        (NODE, "c", {}),
        (NODE, "b", {"label", "Beta!"}),
        (EDGE, "b", "c", {}),
        (EDGE, "a", "b", {"color": "blue"}),
        (ATTR, "bar", "true"),
        ]

min(map(len, data))

3

In [51]:
g = Graph(data)

In [44]:
g.nodes

In [30]:
type([])

list

In [1]:
# triangle

In [ ]:
class TriangleError(Exception):
    pass


class Triangle(object):
    def __init__(self, side_a, side_b, side_c):
        self.validate(side_a, side_b, side_c)
        self._a = side_a
        self._b = side_b
        self._c = side_c
    
    @staticmethod
    def validate(a, b, c):
        if min(a, b, c) <= 0 or max(a, b, c) >= sum([a, b, c]) - max(a, b, c):
            raise TriangleError

    def kind(self):
        uniq_to_kind = {1: 'equilateral', 2: 'isosceles', 3: 'scalene'}
        return uniq_to_kind[len({self._a, self._b, self._c})]
 

In [22]:
t = Triangle(3, 4, 5)
t.kind()

'scalene'

In [15]:
# house

In [104]:
def verse(vn):
    lines = (('the house that Jack built.', ''),
             ('the malt', 'lay in'),
             ('the rat', 'ate'),
             ('the cat', 'killed'),
             ('the dog', 'worried'),
             ('the cow with the crumpled horn', 'tossed'),
             ('the maiden all forlorn', 'milked'),
             ('the man all tattered and torn', 'kissed'),
             ('the priest all shaven and shorn', 'married'),
             ('the rooster that crowed in the morn', 'woke'),
             ('the farmer sowing his corn', 'kept'),
             ('the horse and the hound and the horn', 'belonged to'))
    verse = 'This is '
    for n in range(vn, -1 , -1):
        noun, verb = lines[n]
        verse += noun
        verse += f'\nthat {verb} ' if verb else '\n'
    return verse.strip()


def rhyme():
    return '\n\n'.join([verse(n) for n in range(12)])

In [105]:
print(verse(2))

This is the rat
that ate the malt
that lay in the house that Jack built.


In [106]:
print(rhyme())

This is the house that Jack built.

This is the malt
that lay in the house that Jack built.

This is the rat
that ate the malt
that lay in the house that Jack built.

This is the cat
that killed the rat
that ate the malt
that lay in the house that Jack built.

This is the dog
that worried the cat
that killed the rat
that ate the malt
that lay in the house that Jack built.

This is the cow with the crumpled horn
that tossed the dog
that worried the cat
that killed the rat
that ate the malt
that lay in the house that Jack built.

This is the maiden all forlorn
that milked the cow with the crumpled horn
that tossed the dog
that worried the cat
that killed the rat
that ate the malt
that lay in the house that Jack built.

This is the man all tattered and torn
that kissed the maiden all forlorn
that milked the cow with the crumpled horn
that tossed the dog
that worried the cat
that killed the rat
that ate the malt
that lay in the house that Jack built.

This is the priest all shaven and shor

In [ ]:
# ocr_numbers

In [8]:
num_grid = ["    _  _ ",
            "  | _| _|",
            "  ||_  _|",
            "         ",
            "    _  _ ",
            "|_||_ |_ ",
            "  | _||_|",
            "         ",
            " _  _  _ ",
            "  ||_||_|",
            "  ||_| _|",
            "         ",
            ]

In [93]:
num_grid = [
            "       _     _        _  _ ",
            "  |  || |  || |  |  || || |",
            "  |  ||_|  ||_|  |  ||_||_|",
            "                           "
        ]

In [12]:
def convert(input_grid):
    line_lens_vary = len(set(map(len, input_grid))) != 1
    if line_lens_vary or len(input_grid) % 4 != 0 or len(input_grid[0]) % 3 != 0:
        raise ValueError
    segs_to_digit = {(' _ ', '| |', '|_|', '   '): '0',
                     ('   ', '  |', '  |', '   '): '1',
                     (' _ ', ' _|', '|_ ', '   '): '2',
                     (' _ ', ' _|', ' _|', '   '): '3',
                     ('   ', '|_|', '  |', '   '): '4',
                     (' _ ', '|_ ', ' _|', '   '): '5',
                     (' _ ', '|_ ', '|_|', '   '): '6',
                     (' _ ', '  |', '  |', '   '): '7',
                     (' _ ', '|_|', '|_|', '   '): '8',
                     (' _ ', '|_|', ' _|', '   '): '9'}
    row_ct = len(input_grid)//4
    out = []
    for drn in range(row_ct):
        dig_row = input_grid[drn*4:drn*4+4]
        row_digits_ct = len(dig_row[0])//3
        row = ''
        for rdn in range(row_digits_ct):
            digit = [line[rdn*3:rdn*3+3] for line in dig_row]
            row += segs_to_digit.get(tuple(digit), '?')
        out.append(row)
    return ','.join(out)

In [13]:
convert(num_grid)

'123,456,789'

In [ ]:
# phone_number

In [9]:
class Phone(object):
    def __init__(self, phone_number):
        self._raw = phone_number
        ph = ''.join(ch for ch in phone_number if ch.isdigit())
        ph = ph[1:] if len(ph)==11 and ph[0] == '1' else ph
        if len(ph) != 10 or int(ph[0])<2 or int(ph[3])<2:
            raise ValueError
        self.number = ph
    
    @property
    def area_code(self):
        return self.number[:3]
    
    def pretty(self):
        ph = self.number
        return f'({ph[:3]}) {ph[3:6]}-{ph[6:]}'

ph = Phone('2234567890')
ph, ph.area_code, ph.pretty()

(<__main__.Phone at 0x10ce4e080>, '223', '(223) 456-7890')

In [1]:
# wordy

In [72]:
def split_on_digit(q):
    f = 'X'
    fa = ''
    while f.isalpha():
        fa = fa + ' ' + f
        f, q = q.split(maxsplit=1)
    return fa[3:], int(f), q

def calculate(question):
    q = question.replace('?', ' ?')
    _, operand, rest = split_on_digit(q)
    is_continuing = True
    while is_continuing:
        operator, operand_2, rest = split_on_digit(rest)
        ops_dict = {'plus': operand + operand_2,
                   'minus': operand - operand_2,
                   'multiplied by': operand * operand_2,
                   'divided by': operand / operand_2}
        operand = ops_dict.get(operator, 'ERROR')
        if operand == 'ERROR':
            raise ValueError
        if rest == '?':
            is_continuing = False
    return operand

In [73]:
question = "What is -333 divided by 3 plus 1 minus -33?"
calculate(question)

-77.0

In [ ]:
# queen attack

In [ ]:
def board(white_position, black_position):
    board = [['_']*8]*8
    for (r, c), ch in zip((white_position, black_position), ('W', 'B')):
        if min(r, c) < 0 or max(r, c) > 7 or white_position==black_position:
            raise ValueError
        rm = board[r][:]
        rm[c] = ch
        board[r] = rm
    return list(map(lambda r: ''.join(r), board))


def can_attack(white_position, black_position):
    wr, wc = white_position
    br, bc = black_position
    if min(wr, wc, br, bc) < 0 or max(wr, wc, br, bc) > 7 or (wr, wc)==(br, bc):
        raise ValueError
    return wr==br or wc==bc or abs(wr-br)==abs(wc-bc)

In [ ]:
board((2, 3), (5, 6))

In [ ]:
# minesweeper

In [ ]:
def star_ct(r, c, inp):
    row_ct = len(inp)
    col_ct = len(inp[0])
    stars = 0
    for ri in [r-1, r, r+1]:
        for ci in [c-1, c, c+1]:
            if 0 <= ri < row_ct and 0 <= ci < col_ct:
                if inp[ri][ci] == '*':
                    stars += 1
    return str(stars) if stars else ' '

def board(input_board_array):
    if len(set(map(len, input_board_array))) > 1:
        raise ValueError('rows are not of equal length')
    out = []
    for r, row in enumerate(input_board_array):
        out_row = []
        for c, val in enumerate(row):
            if val == ' ':
                out_row.append(star_ct(r, c, input_board_array))
            elif val == '*':
                out_row.append('*')
            else:
                raise ValueError('invalid char encountered')
        out.append(''.join(out_row))
    return out

In [ ]:
inp = ["***", "* *", "***"]
board(inp)

In [ ]:
inp = [" * * "]
board(inp)

In [ ]:
inp = [" ",
       "*",
       " ",
       "*",
       " "]
board(inp)

In [ ]:
inp = ["*",
       " ",
       "  ",
       " ",
       "*"]
board(inp)

In [ ]:
len(set(map(len, inp))) > 1

In [ ]:
# check_brackets

In [ ]:
def check_brackets(input_string):
    br_opening = '{[('
    br_closing = '}])'
    br_closes = dict(zip(br_closing, br_opening))
    br_stack = []
    for ch in input_string:
        if ch in br_opening:
            br_stack.append(ch)
        if ch in br_closing:
            if br_stack and br_closes[ch] == br_stack[-1]:
                br_stack.pop()
            else:
                return False
    return br_stack == []
        


check_brackets("[({]})"), check_brackets("([{}({}[])])")